# Các thư viện cần thiết

In [1]:
%pip install unidecode

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import time
from unidecode import unidecode

In [3]:
# Đường dẫn đến ChromeDriver
DRIVER_PATH = 'C:\\chromedriver\\chromedriver.exe'

# Tạo đối tượng Service
service = Service(DRIVER_PATH)

# Khởi tạo ChromeDriver với service
driver =None



In [4]:
def scroll_and_load_more(driver, max_scrolls=3):
    scroll_count = 0
    while scroll_count < max_scrolls:
        # Cuộn xuống cuối trang
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Chờ trang tải đầy đủ
        
        # Tìm nút "Tải thêm" và nhấp vào nếu có
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-primary"))
            )
            actions = ActionChains(driver)
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            time.sleep(0.5)  # Chờ để cuộn đến nút "Tải thêm"

            # Cuộn trang thêm một đoạn để đảm bảo nút không bị che khuất
            driver.execute_script("window.scrollBy(0, -150);")
            time.sleep(0.5)  # Chờ để cuộn

            actions.move_to_element(load_more_button).click().perform()
            time.sleep(3)  # Chờ để tải thêm dữ liệu
        except Exception as e:
            print(f"Exception encountered: {e}")
            break
        
        scroll_count += 1


# Cào link

In [6]:
##danh sách để lưu trữ các link
driver = webdriver.Chrome(service=service)
driver.get("https://guland.vn/mua-ban-bat-dong-san")
scroll_and_load_more(driver, max_scrolls=20)

# Danh sách để lưu trữ các link
arr = []
linkSelector = ".sdb-image-wrap "  # CSS selector cho các thẻ <a> trong sdb-image-wrap

# Lấy các liên kết
links = driver.find_elements(By.CSS_SELECTOR, linkSelector)
for link in links:
    arr.append(link.get_attribute('href'))

# Tạo DataFrame từ danh sách các link
df = pd.DataFrame(arr, columns=['links'])
print("Hoàn thành lấy dữ liệu!")
print(df)

# Đóng trình duyệt
driver.quit()


Hoàn thành lấy dữ liệu!
                                                 links
0    https://guland.vn/post/mat-tien-kinh-doanh-tra...
1       https://guland.vn/post/an-cu-lap-nghiep-926345
2    https://guland.vn/post/ban-nha-di-an-gia-re-nh...
3    https://guland.vn/post/ban-dat-339m2-33-ty-tai...
4    https://guland.vn/post/0938113403-dat-full-100...
..                                                 ...
940  https://guland.vn/post/ban-dat-1305m2-3-ty-tai...
941  https://guland.vn/post/ban-dat-van-phat-hung-p...
942  https://guland.vn/post/dat-mat-tien-huynh-thuc...
943  https://guland.vn/post/dat-mat-tien-ngo-duc-to...
944  https://guland.vn/post/can-ban-dat-tho-cu-mat-...

[945 rows x 1 columns]


# Xuất file link

In [7]:
df.to_csv('data\\guland_links.csv', index=False)

# Check xem dữ liệu có bị trùng lặp không

In [8]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
940    False
941    False
942    False
943    False
944    False
Length: 945, dtype: bool

# Test thử xem có lấy dữ liệu không

In [4]:
frame = pd.read_csv('data\\guland_links.csv')
frame.shape
frame.head(20)

,links
0,https://guland.vn/post/mat-tien-kinh-doanh-tra...
1,https://guland.vn/post/an-cu-lap-nghiep-926345
2,https://guland.vn/post/ban-nha-di-an-gia-re-nh...
3,https://guland.vn/post/ban-dat-339m2-33-ty-tai...
4,https://guland.vn/post/0938113403-dat-full-100...
5,https://guland.vn/post/dat-biet-thu-mat-tien-s...
6,https://guland.vn/post/ban-dat-100m2-780-trieu...
7,https://guland.vn/post/ban-can-ho-soho-residen...
8,https://guland.vn/post/ban-dat-724m2-42-ty-tai...
9,https://guland.vn/post/dat-thu-dau-1-binh-duon...


# Cào dữ liệu

In [5]:
dia_chi=[]
ma_tin=[]
gia=[]
dien_tich=[]
gia_tren_m2=[]
ten_du_an=[]
loai_bds=[]
chieu_ngang,chieu_dai=[],[]
so_phong_ngu,so_phong_tam=[],[]
so_tang=[]
vi_tri,huong_cua=[],[]
duong_hem,loai_duong=[],[]
items =['Tên dự án:','Loại BĐS:','Chiều ngang:','Chiều dài:','Số phòng ngủ:','Số phòng tắm:',
        'Số tầng','Vị trí:',"Hướng cửa chính",'Đường/hẻm vào rộng','Loại đường'
        ]

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from requests_html import HTML
def getdata(link) ->dict:
    # Cấu hình Selenium WebDriver
    # Tạo instance của trình điều khiển Chrome
    driver = webdriver.Chrome(service=service)
        # Mở trang web        
    driver.get(link)
        # Lấy nội dung HTML của trang
    html_text = driver.page_source
    driver.quit()  # Đóng trình duyệt sau khi lấy dữ liệu
        # Phân tích cú pháp HTML để trích xuất dữ liệu
    r_html = HTML(html=html_text)
    if len(r_html.find(".notfound-message")) == 0:
        arr = []
        r_table = r_html.find(".sdb-picker-site")
        if len(r_table) > 0:
            # test là toàn bộ mã html của trang đang truy xuất
            test = r_table[0]
            main_info=test.find('.dtl-prc__row')
            tmp=test.find('.dtl-stl__row')
            dia_chi=tmp[0]
            ma_tin_tmp = test.find('.dtl-stl__row')[1]
            ma_tin_text = ma_tin_tmp.text
            
            # Tìm vị trí của chuỗi "Mã tin:" để lấy mã tin
            start_index = ma_tin_text.find(":")
            end_index= ma_tin_text.find("Cập nhật")
            ma_tin = ma_tin_text[start_index+2 :end_index].strip()
            tmp1=test.find('.dtl-inf__row')
            inf = tmp1[0].text
            arr.append(main_info[0].text)  # các thông tin chính các căn nhà, gồm có tiền, diện tích
            arr.append(dia_chi.text)# địa chỉ của căn nhà
            arr.append(ma_tin)# mã tin
            arr.append(inf)# các thông tin phụ của căn nhà
        else:
            return []
        str1 = arr
        str1 = [arr]
        temp = arr[0].split('\n')#tách dữ liệu chính
        tmp=arr[3].split('\n')# tách các dữ liệu phụ
        idx = []
        idx.append(arr[1])
        idx.append(arr[2])
        idx.append(temp[0])#gia
        idx.append(temp[1])#dien tich
        idx.append(temp[2])#gia tren m^2
        for i in range(0, len(tmp), 2):
            if i + 1 < len(tmp):
                idx.append([tmp[i]+" "+ tmp[i + 1]])
            else:
                idx.append([tmp[i]])
        return idx
    else:
        return []




# Cào và lưu json

In [7]:
import json

with open('data\\guland_data.json', 'w', encoding='utf-8') as file:
    file.write('[')  # Bắt đầu một json array
    first = True
    
    for link in frame['links']:
        try:
            print(f"Processing link: {link}")
            data = getdata(link)
            dem = 0   
            if len(data) > 0:
                # Tạo dictionary để lưu dữ liệu cho mục hiện tại
                entry = {
                    'dia_chi': data[0],
                    'ma_tin': data[1],
                    'gia': data[2],
                    'dien_tich': data[3],
                    'gia_tren_m2': data[4]
                }
                
                for item in items:
                    mang = []
                    flag = 0
                    for i in data:
                        if len(i) > 0 and item in i[0]:
                            mang.append(i[0].split(':')[1])
                            flag = 1
                            break
                    dem += 1
                    key = None
                    if dem == 1: key = 'ten_du_an'
                    elif dem == 2: key = 'loai_bds'
                    elif dem == 3: key = 'chieu_ngang'
                    elif dem == 4: key = 'chieu_dai'
                    elif dem == 5: key = 'so_phong_ngu'
                    elif dem == 6: key = 'so_phong_tam'
                    elif dem == 7: key = 'so_tang'
                    elif dem == 8: key = 'vi_tri'
                    elif dem == 9: key = 'huong_cua'
                    elif dem == 10: key = 'duong_hem'
                    elif dem == 11: key = 'loai_duong'
                    
                    if key:
                        if flag == 1:
                            entry[key] = mang[0] if mang else None
                        else:
                            entry[key] = None
                
                # Viết entry vào file JSON
                if not first:
                    file.write(',\n')
                json.dump(entry, file, ensure_ascii=False, indent=4)
                first = False
        except Exception as e:
            print(f"Error processing link: {link}")
            print(f"Exception: {e}")

    file.write(']')


Processing link: https://guland.vn/post/mat-tien-kinh-doanh-tran-hung-dao-di-an-bd-926353
Processing link: https://guland.vn/post/an-cu-lap-nghiep-926345
Processing link: https://guland.vn/post/ban-nha-di-an-gia-re-nha-cap-4-binh-duong-926299
Processing link: https://guland.vn/post/ban-dat-339m2-33-ty-tai-phuong-ninh-son-thanh-pho-tay-ninh-926259
Processing link: https://guland.vn/post/0938113403-dat-full-100-tho-cu-10x20-phuong-ninh-ha-cach-ql1a-50m-926089
Processing link: https://guland.vn/post/dat-biet-thu-mat-tien-suong-nguyet-anh-dong-hoa-di-an-926226
Processing link: https://guland.vn/post/ban-dat-100m2-780-trieu-tai-xa-ea-tu-thanh-pho-buon-ma-thuot-875361
Processing link: https://guland.vn/post/ban-can-ho-soho-residence-quan-1-noi-that-dep-918539
Processing link: https://guland.vn/post/ban-dat-724m2-42-ty-tai-xa-an-nhon-tay-huyen-cu-chi-926046
Processing link: https://guland.vn/post/dat-thu-dau-1-binh-duong-925951
Processing link: https://guland.vn/post/ban-nha-1tret2lau-binh-nh

# Đọc file json

In [8]:
with open('data\\guland_data.json', 'r',encoding='utf-8') as file:
    json_data = json.load(file)

df_items = pd.json_normalize(json_data)
df_items

,dia_chi,ma_tin,gia,dien_tich,gia_tren_m2,ten_du_an,loai_bds,chieu_ngang,chieu_dai,so_phong_ngu,so_phong_tam,so_tang,vi_tri,huong_cua,duong_hem,loai_duong
0,"Phường Đông Hòa, Thị xã Dĩ An, Bình Dương",926353,7.8 tỷ,112m²,69.64 triệu /m²,None,"Đất ở, đất thổ cư",5m,None,None,None,None,Đường chính,None,16m,Đường nhựa
1,"Phường Phú Lợi, Thành phố Thủ Dầu Một, Bình Dương",926345,2.79 tỷ,141m²,19.79 triệu /m²,None,"Đất ở, đất thổ cư",5.7m,25m,None,None,None,None,None,4m,Đường nhựa
2,"Phường Tân Đông Hiệp, Thị xã Dĩ An, Bình Dương",926299,2.35 tỷ,68m²,34.56 triệu /m²,None,"Đất ở, đất thổ cư",4m,None,None,None,None,Trong hẻm,None,3m,Đường bê tông
3,"Phường Ninh Sơn, Thành phố Tây Ninh, Tây Ninh",926259,3.3 tỷ,339m²,9.73 triệu /m²,None,đất,10m,None,None,None,None,None,None,None,None
4,"Phường Ninh Hà, Thị xã Ninh Hòa, Khánh Hòa",926089,939 triệu,200m²,4.7 triệu /m²,None,"Đất ở, đất thổ cư",10m,20m,None,None,None,None,None,4m,Đường đất
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,"Thị trấn Trới, Thành phố Hạ Long, Quảng Ninh",C14 - 1,3 tỷ,130.5m²,22.99 triệu /m²,None,Đất nền dự án,9m,None,None,None,None,Đường chính,Tây Bắc,7.5m,Đường bê tông
914,"đường 15B Đường Nguyễn Lương Bằng, Xã Phú Xuân...",923120,5.5 tỷ,126m²,43.65 triệu /m²,None,Đất nền dự án,5m,25m,None,None,None,Đường chính,None,12m,Đường nhựa
915,"Đường Huỳnh Thúc Kháng, Phường Hàm Tiến, Thành...",901400,13.8 tỷ,536m²,25.75 triệu /m²,None,"Đất TMDV, sản xuất",37m,None,None,None,None,Đường chính,Đông,30m,Đường nhựa
916,"Đường Ngô Đức Tốn, Xã Hàm Liêm, Huyện Hàm Thuậ...",900589,4.6 tỷ,526m²,8.75 triệu /m²,None,"Đất ở, đất thổ cư",9.99m,None,None,None,None,Đường chính,None,10m,Đường nhựa
